# Building an intelligent LLM Routing System using CrewAI

This notebook demonstrates how to build an LLM workflow using [CrewAI](https://docs.crewai.com/) for intelligent routing. 

## Use Case
Sales and marketing teams face diverse processing needs—from creative content generation to data-driven analysis to customer communications. Each of these tasks benefits from models with different strengths. Rather than using a one-size-fits-all approach, our routing system will analyze incoming requests, categorize them, and direct them to specialized models that excel at specific tasks.

Using langgraph, we'll be enable an LLM workflow that perform the following:

- Classify incoming requests by type and intent
- Create a decision framework that maps request categories to appropriate models
- Implement routing logic that directs requests to the optimal model

The following diagram depicts the overall architecture and the worklfow:

<img src="../../imgs/lab3-prompt-routing-architecture.png" width="800"> 

## Environment Setup

First, we'll load the environment variables and import the necessary libraries for building our CrewAI flow.

In [2]:
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel
from crewai import LLM

## State Management

The `AgentState` class defines the data structure that will be passed between different steps in our flow. This ensures consistent state management throughout the routing process.

In [20]:
class AgentState(BaseModel):
    user_query: str = None
    task_type: str = ""

## RouterFlow Implementation

This class implements the core routing logic with three main components:

1. **Classification Step** (`start_method`): Analyzes the user query and determines the task type
2. **Routing Logic** (`router_step`): Directs the flow to the appropriate specialized handler  
3. **Specialized Handlers**: Execute task-specific processing using optimized models which handles 2 different task types: `strategy` and `social_media`

In [21]:
import random 

class RouterFlow(Flow[AgentState]):

    @start()
    def start_method(self):
        print("Starting the Flow")
        system_prompt = """You are a specialized routing assistant for a Media & Entertainment marketing team. Your job is to classify incoming marketing requests into one of these categories: 1. STRATEGY - Questions about developing comprehensive marketing strategies, campaign planning, narratives, audience targeting, marketing analytics, ROI assessment, or long-term marketing plans 2. SOCIAL_MEDIA - Requests for creating social media posts, captions, hashtags, or content for specific platforms. Only respond with exactly one category label: either "strategy" or "social_media"."""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": self.state.user_query}
        ]
        llm = LLM(model="bedrock/us.amazon.nova-premier-v1:0")
        response = llm.call(messages=messages)
        self.state.task_type = response.lower()

    @router(start_method)
    def router_step(self):
        print("Router Step")
        if self.state.task_type == "strategy":
            return "strategy"
        elif self.state.task_type == "social_media":
            return "social_media"

    @listen("strategy")
    def strategy_step(self):
        print("Strategy Step")
        llm = LLM(model="us.anthropic.claude-3-5-sonnet-20241022-v2:0")
        strategy_prompt = f"You are a marketing strategy expert for Media & Entertainment companies. Please develop a comprehensive strategic response to this request:{self.state.user_query}. Include actionable insights on audience targeting, campaign planning, and ROI measurement."
        response = llm.call(strategy_prompt)
        return response
    
        
    @listen("social_media")
    def social_media_step(self):
        print("Social Media Step")
        llm = LLM(model="us.amazon.nova-lite-v1:0")
        social_media_prompt = f"""You are a social media expert for Media & Entertainment companies. Create engaging social media content for this request:{self.state.user_query}. Include creative captions, relevant hashtags, and platform-specific tips."""
        response = llm.call(social_media_prompt)
        return response

## Testing the Router

Let's test our routing system with a strategy-focused query to see how it classifies and processes the request through the appropriate workflow.

In [ ]:
user_query = """We need to analyze the ROI of our summer blockbuster campaign across different marketing channels. Can you develop a framework for measuring effectiveness and determining which platforms delivered the best returns?"""
response = await RouterFlow().kickoff_async(inputs = { "user_query" : user_query})
print(response)

## Key Benefits of This Approach

- **Intelligent Routing**: Automatically directs requests to models optimized for specific tasks
- **Cost Optimization**: Uses lighter models for simpler tasks, more powerful models for complex analysis
- **Scalability**: Easy to add new categories and specialized handlers
- **Maintainability**: Clear separation between routing logic and task execution